In [1]:
from pandas import read_fwf
import numpy as np
import pandas as pd

In [35]:
# This script reads data files from the DHS survey
# https://dhsprogram.com/data/dataset/Zambia_Standard-DHS_2018.cfm
# and then outputs selected columns to a csv file


data_folder = '../Data/DHSSurvey/'
employee_survey_folder = 'DHSservices/KEIR8CDT/'
# Choose the file prefix.
filename = 'KEIR8CFL'
filepath = data_folder + employee_survey_folder + filename + '.DTA'

In [36]:
# When labels are repeated, use the method below https://stackoverflow.com/questions/31782283/loading-stata-file-categorial-values-must-be-unique
with pd.io.stata.StataReader(filepath) as sr:
    value_labels = sr.value_labels()

df = pd.read_stata(
    filepath,
    convert_categoricals=False,
)

for col in value_labels:
    if col.lower() in df.columns:
        df[col.lower()].replace(value_labels[col], inplace=True)

C:\Users\rheredia\AppData\Local\Temp\ipykernel_22560\708482035.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col.lower()].replace(value_labels[col], inplace=True)


In [4]:
# df

In [37]:
# label variable v005     "Women's individual sample weight (6 decimals)"
# label variable v024     "Region"
# label variable v025     "Type of place of residence"
# label variable v704     "Husband/partner's occupation"
# label variable v704a    "Husband/partner worked in last 7 days/12 months"
# label variable v705     "Husband/partner's occupation (grouped)"
# label variable v714     "Respondent currently working"
# label variable v714a    "Respondent has a job, but currently absent"
# label variable v716     "Respondent's occupation"
# label variable v717     "Respondent's occupation (grouped)"
# label variable v719     "Respondent works for family, others, self"
# label variable v721     "NA - Respondent works at home or away"
labels_services = ['v005','v024', 'v025', 'v704', 'v704a', 'v705', 'v714', 'v714a', 'v716','v717', 'v719', 'v721']
df_services = df[labels_services]

Check encoding

In [38]:
import chardet

file_path = data_folder + employee_survey_folder + filename + '.DO'

# Detect encoding
with open(file_path, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)  # Detect encoding
    encoding = result['encoding']
    print("Detected Encoding:", encoding)  # Check the detected encoding

Detected Encoding: utf-8


In [39]:
# Read the names of the columns from the .DO file and convert to dictionary
do = pd.read_fwf(data_folder + employee_survey_folder + filename +'.DO',
               skiprows=2, encoding='utf-8')

In [40]:
col_dict = dict(zip(do['caseid'], do['"Case Identification"']))

In [41]:
df_services = df_services.rename(columns=col_dict)
df_services.head(5)

,"""Women's individual sample weight (6 decimals)""","""Region""","""Type of place of residence""","""Husband/partner's occupation""","""Husband/partner worked in last 7 days/12 months""","""Husband/partner's occupation (grouped)""","""Respondent currently working""","""Respondent has a job, but currently absent""","""Respondent's occupation""","""Respondent's occupation (grouped)""","""Respondent works for family, others, self""","""NA - Respondent works at home or away"""
0,1296049,mombasa,urban,decorators and other commercial workers,worked last 7 days,professional/technical/managerial,no,no,not working and didn't work in last 12 months,not working,NaN,NaN
1,1296049,mombasa,urban,motor vehicle drivers,worked last 7 days,skilled manual,no,no,cleaners launders and domestic workers,household and domestic,for someone else,NaN
2,1296049,mombasa,urban,NaN,NaN,NaN,yes,NaN,non-departmental managers,professional/technical/managerial,self-employed,NaN
3,1296049,mombasa,urban,street venders and related workers,worked last 7 days,unskilled manual,yes,NaN,computing professionals,professional/technical/managerial,for someone else,NaN
4,1296049,mombasa,urban,motor vehicle drivers,worked last 7 days,skilled manual,yes,NaN,computing professionals,professional/technical/managerial,for someone else,NaN


In [42]:
# Calculate the occupation share within each region and per type of area - women
occupation_share_women = df.groupby(['v024', 'v025', 'v717'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_women = occupation_share_women.div(occupation_share_women.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_women = occupation_share_women.applymap('{:.2f}'.format)
occupation_share_women

C:\Users\rheredia\AppData\Local\Temp\ipykernel_22560\1970664557.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  occupation_share_women = occupation_share_women.applymap('{:.2f}'.format)


v717             agricultural - employee agricultural - self employed  \
v024       v025                                                         
baringo    rural                   18.74                         1.57   
           urban                    7.59                         1.02   
bomet      rural                   42.51                         2.38   
           urban                   11.34                         0.00   
bungoma    rural                   19.04                         0.18   
...                                  ...                          ...   
vihiga     urban                    6.02                         0.00   
wajir      rural                    0.00                         0.26   
           urban                    1.95                         0.63   
west pokot rural                   11.37                         0.40   
           urban                    0.00                         0.00   

v717             clerical don't know household and domestic not working  \
v024       v025                                                           
baringo    rural     0.00       0.97                   8.52       35.07   
           urban     1.35       2.73                   9.33       20.30   
bomet      rural     0.54       0.64                   2.73       31.55   
           urban     2.56       5.74                   5.14       26.70   
bungoma    rural     0.66       0.89                   3.26       38.05   
...                   ...        ...                    ...         ...   
vihiga     urban     1.62       0.54                   4.37       51.65   
wajir      rural     0.29       0.12                   0.15       90.82   
           urban     0.00       0.93                   3.74       75.54   
west pokot rural     0.19       0.00                   0.38       76.22   
           urban     2.37       0.93                   5.30       56.10   

v717             professional/technical/managerial  sales services  \
v024       v025                                                      
baringo    rural                             20.75   3.16     3.36   
           urban                             27.94   7.47     9.67   
bomet      rural                              8.90   3.35     2.18   
           urban                             18.22  13.92     9.75   
bungoma    rural                             12.93   4.35     5.47   
...                                            ...    ...      ...   
vihiga     urban                             14.68   6.30     6.03   
wajir      rural                              4.94   1.83     0.65   
           urban                              7.78   1.57     3.56   
west pokot rural                              6.91   2.90     1.19   
           urban                             17.92  12.04     2.95   

v717             skilled manual unskilled manual  
v024       v025                                   
baringo    rural           0.42             7.42  
           urban           2.02            10.58  
bomet      rural           1.02             4.21  
           urban           0.00             6.64  
bungoma    rural           0.78            14.39  
...                         ...              ...  
vihiga     urban           0.00             8.80  
wajir      rural           0.46             0.48  
           urban           0.35             3.96  
west pokot rural           0.32             0.13  
           urban           0.20             2.18  

[92 rows x 11 columns]

In [43]:
# Calculate the occupation share within each region - women - to compare with table 3.7.1 from the DHS report
occupation_share_women_DHSreport = df.groupby(['v024', 'v717'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_women_DHSreport = occupation_share_women_DHSreport.drop(['not working'], axis=1)
occupation_share_women_DHSreport = occupation_share_women_DHSreport.div(occupation_share_women_DHSreport.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_women_DHSreport = occupation_share_women_DHSreport.applymap('{:.2f}'.format)
occupation_share_women_DHSreport

C:\Users\rheredia\AppData\Local\Temp\ipykernel_22560\2755512958.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  occupation_share_women_DHSreport = occupation_share_women_DHSreport.applymap('{:.2f}'.format)


v717,agricultural - employee,agricultural - self employed,clerical,don't know,household and domestic,professional/technical/managerial,sales,services,skilled manual,unskilled manual
v024,,,,,,,,,,
baringo,25.03,2.20,0.34,1.88,12.84,32.58,5.76,6.55,1.03,11.79
bomet,60.55,3.37,0.88,1.16,4.08,13.40,5.36,3.52,1.44,6.25
bungoma,27.50,0.26,1.08,1.42,5.59,23.35,7.93,8.96,1.34,22.58
busia,44.85,0.99,0.53,0.56,5.82,17.30,6.54,9.72,3.19,10.48
elgeyo-marakwet,44.64,1.97,0.67,0.00,2.67,33.03,6.42,2.85,0.00,7.76
embu,38.80,0.63,2.51,2.00,6.47,38.32,1.73,6.87,0.96,1.71
garissa,0.62,0.00,1.70,4.67,20.17,26.33,14.96,7.11,5.38,19.06
homa bay,30.29,0.00,0.83,1.72,8.29,26.96,14.66,6.64,2.90,7.72
isiolo,7.79,1.76,2.36,0.51,7.19,36.05,13.12,4.66,0.91,25.65


In [44]:
# Calculate the share of each occupation within each region and per type of area = men
occupation_share_men = df.groupby(['v024', 'v025', 'v705'])['v005'].sum().unstack(fill_value=0)/1000000
occupation_share_men = occupation_share_men.div(occupation_share_men.sum(axis=1), axis=0) * 100  # Calculate share as percentage
occupation_share_men = occupation_share_men.applymap('{:.2f}'.format)
occupation_share_men

C:\Users\rheredia\AppData\Local\Temp\ipykernel_22560\774788351.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  occupation_share_men = occupation_share_men.applymap('{:.2f}'.format)


v705             agricultural - employee agricultural - self employed  \
v024       v025                                                         
baringo    rural                   27.59                         2.62   
           urban                    7.45                         1.98   
bomet      rural                   38.95                         1.63   
           urban                    9.37                         0.00   
bungoma    rural                   28.94                         0.34   
...                                  ...                          ...   
vihiga     urban                    9.96                         0.00   
wajir      rural                   10.01                         2.44   
           urban                    6.31                         0.94   
west pokot rural                   10.13                         0.00   
           urban                    3.84                         0.00   

v705             clerical did not work don't know household and domestic  \
v024       v025                                                            
baringo    rural     0.00         9.05       1.12                   5.09   
           urban     1.26         2.24       2.33                   1.60   
bomet      rural     0.29         2.30       0.98                   3.28   
           urban     4.96         0.00       0.00                   2.00   
bungoma    rural     0.00         4.54       1.96                   2.21   
...                   ...          ...        ...                    ...   
vihiga     urban     0.00         4.60       2.96                   2.05   
wajir      rural     0.00        55.98       1.63                   0.77   
           urban     1.67        25.73       1.75                   0.00   
west pokot rural     0.56        58.37       1.54                   0.56   
           urban     1.72        13.66       3.19                   1.15   

v705             professional/technical/managerial  sales services  \
v024       v025                                                      
baringo    rural                             19.20   6.23     9.01   
           urban                             29.57   8.61     6.72   
bomet      rural                             17.04   2.95     7.66   
           urban                             41.52   9.10     1.24   
bungoma    rural                             16.19   5.34     5.21   
...                                            ...    ...      ...   
vihiga     urban                             26.93   7.51     4.32   
wajir      rural                             11.18   4.43     1.31   
           urban                             28.89   3.87     1.56   
west pokot rural                             12.90   4.70     1.72   
           urban                             33.99  14.60     2.18   

v705             skilled manual unskilled manual  
v024       v025                                   
baringo    rural          16.60             3.48  
           urban          25.41            12.81  
bomet      rural          21.62             3.29  
           urban          24.78             7.03  
bungoma    rural          24.47            10.79  
...                         ...              ...  
vihiga     urban          20.39            21.27  
wajir      rural           9.34             2.90  
           urban          20.03             9.25  
west pokot rural           7.55             1.97  
           urban          16.74             8.91  

[92 rows x 11 columns]

In [45]:
# Save the two DataFrame to a CSV file
occupation_share_men.to_csv(data_folder + 'employee_survey_men.csv')
occupation_share_women.to_csv(data_folder + 'employee_survey_women.csv')